In [1]:
# data model format for GPT
# ROle specific
# we have to prepara training data with few examples
# single sample data 1 data of example
# {
#     "messages": [
#         { "role": "system" , "content": "my hotel bot which is helping customer untill they are not satified" },
#         { "role": "user" , "content": "i can't find wifi password" },
#         { "role": "assistant" , "content": "hey sorry for inconvience but wifi passsword you have to just ask at reception" }
#     ],
#     "messages": [
#         { "role": "system" , "content": "my hotel bot which is helping customer untill they are not satified" },
#         { "role": "user" , "content": "i need to order food where is the menu " },
#         { "role": "assistant" , "content": "there must be a table in you room check there else please call 9 extension" }
#     ]
# }
!pip install openai==0.28
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


In [6]:
import csv
import json
import openai
import numpy as np
from collections import defaultdict

In [28]:
# load api Key
openai.api_key = ""

In [4]:
# loading and cleaning of data
ashudata_path = "/content/sample_data/hotel_hospitality_data.csv"

cleaned_data = []

with open(ashudata_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

# printing clean data
#print(cleaned_data[0])
# save data into jsonl format
ashudata_jsonl_path = "/content/sample_data/ashuhotel_hospitality_data.jsonl"
with open(ashudata_jsonl_path,'w',encoding='utf-8-sig') as f1:
  for data in cleaned_data:
    f1.write(json.dumps(data) + '\n')

In [5]:
# take 2 or 3 minutes then drop done message
# validate /
ashudata_jsonl_path1 = "/content/sample_data/ashuhotel_hospitality_data.jsonl"

with open(ashudata_jsonl_path1, 'r',encoding='utf-8-sig') as f2:
   mydataset = [json.loads(i) for i in f2]

# checking lengh
print("mydataset length is ",len(mydataset))
print("first example ")
for i in mydataset[0]["messages"]:
  print(i)

mydataset length is  18
first example 
{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}
{'role': 'user', 'content': 'Where is the gym located?'}
{'role': 'assistant', 'content': "Our gym is located on the 2nd floor. I'll be happy to show you the way if you need help!"}


In [7]:
# format validation
# Format error checks
format_errors = defaultdict(int)

for ex in mydataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [9]:
# upload validated to gpt hosted server
ashu_training = openai.File.create(
    file=open(ashudata_jsonl_path1,"rb"),
    purpose="fine-tune")
# display details
print(ashu_training)
# pick file id
ashu_file_id = ashu_training["id"]
print("ashutoshh fine tune data file id is ",ashu_file_id)

{
  "object": "file",
  "id": "file-pAewQamlwNemgGd1DG8JyL3S",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 7555,
  "created_at": 1723007689,
  "status": "processed",
  "status_details": null
}
ashutoshh fine tune data file id is  file-pAewQamlwNemgGd1DG8JyL3S


In [14]:
# fine tuning model with custom uploaded data
# GPT models -- are limited
ashu_model_name = "ashu_walmartbot"
ashu_model_response = openai.FineTuningJob.create(
    training_file=ashu_file_id,
    suffix=ashu_model_name,
    model="gpt-3.5-turbo"
)
# printing fine tuned model id
print(ashu_model_response)

ashu_fm_final_model_id = ashu_model_response["id"]
print(ashu_fm_final_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-KiAK2Ap5SVrV66Z5mBQEAPSM",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1723011271,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-sW9LYXODuBOJUwL7Lo4RNLZu",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-pAewQamlwNemgGd1DG8JyL3S",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "ashu_walmartbot",
  "seed": 239882505,
  "estimated_finish": null,
  "integrations": []
}
ftjob-KiAK2Ap5SVrV66Z5mBQEAPSM


In [21]:
# listing job process
ashu_process = openai.FineTuningJob.list_events(id=ashu_fm_final_model_id ,limit=50)
ashu_events = ashu_process["data"]
ashu_events.reverse()
for msg in ashu_events:
  print(msg["message"])

Step 46/90: training loss=0.17
Step 47/90: training loss=0.43
Step 48/90: training loss=0.34
Step 49/90: training loss=0.39
Step 50/90: training loss=0.43
Step 51/90: training loss=0.14
Step 52/90: training loss=0.22
Step 53/90: training loss=0.14
Step 54/90: training loss=0.39
Step 55/90: training loss=0.50
Step 56/90: training loss=0.22
Step 57/90: training loss=0.36
Step 58/90: training loss=0.09
Step 59/90: training loss=0.16
Step 60/90: training loss=0.25
Step 61/90: training loss=0.80
Step 62/90: training loss=0.28
Step 63/90: training loss=0.21
Step 64/90: training loss=0.05
Step 65/90: training loss=0.07
Step 66/90: training loss=0.39
Step 67/90: training loss=0.22
Step 68/90: training loss=0.19
Step 69/90: training loss=0.07
Step 70/90: training loss=0.50
Step 71/90: training loss=0.14
Step 72/90: training loss=0.12
Step 73/90: training loss=0.32
Step 74/90: training loss=0.45
Step 75/90: training loss=0.04
Step 76/90: training loss=0.66
Step 77/90: training loss=0.03
Step 78/

In [35]:
# testing model
ashu_test_message = []
# refering system message

system_message = "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."
ashu_test_message.append({ "role": "system" , "content": system_message})
user_input = "hey i am unable to find wifi password"

ashu_test_message.append({ "role": "user" , "content": user_input})

# printing input
print(ashu_test_message)

[{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}, {'role': 'user', 'content': 'hey i am unable to find wifi password'}]


In [31]:
# retrive model id
ashu_id_response = openai.FineTuningJob.retrieve(ashu_fm_final_model_id)
print(ashu_id_response)
# geting fine tune model id
real_ashu_model_finetID = ashu_id_response["fine_tuned_model"]
print("my fine tuned model id is ",real_ashu_model_finetID)
print(type(real_ashu_model_finetID))



{
  "object": "fine_tuning.job",
  "id": "ftjob-KiAK2Ap5SVrV66Z5mBQEAPSM",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1723011271,
  "finished_at": 1723012530,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tUYeBXK",
  "organization_id": "org-sW9LYXODuBOJUwL7Lo4RNLZu",
  "result_files": [
    "file-lXUab6SJA3fYxerKXK9zEHbn"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-pAewQamlwNemgGd1DG8JyL3S",
  "hyperparameters": {
    "n_epochs": 5,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 6695,
  "error": {},
  "user_provided_suffix": "ashu_walmartbot",
  "seed": 239882505,
  "estimated_finish": null,
  "integrations": []
}
my fine tuned model id is  ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tUYeBXK
<class 'str'>


In [32]:
response = openai.Model.list()

for model in response['data']:
    print(model['id'])

dall-e-3
gpt-4-1106-preview
dall-e-2
tts-1-hd-1106
tts-1-hd
text-embedding-3-large
gpt-4-0125-preview
babbage-002
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
text-embedding-3-small
tts-1
gpt-3.5-turbo
whisper-1
gpt-4o-2024-08-06
text-embedding-ada-002
gpt-3.5-turbo-16k
davinci-002
gpt-4-turbo-2024-04-09
tts-1-1106
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4-0613
gpt-4
ft:gpt-3.5-turbo-0125:personal:chatner-bot:9t3xDPIC:ckpt-step-54
ft:gpt-3.5-turbo-0125:personal:chatner-bot:9t3xDQOW:ckpt-step-72
ft:gpt-3.5-turbo-0125:personal:chatner-bot:9t3xDM1s
ft:gpt-3.5-turbo-0125:personal:ashu-hotel-bot:9tJBnwBd:ckpt-step-54
ft:gpt-3.5-turbo-0125:personal:sal-hotel-bots:9tJHvgu0:ckpt-step-72
ft:gpt-3.5-turbo-0125:personal:mg-hotel-bot:9tJ5jMgn:ckpt-step-54
ft:gpt-3.5-turbo-0125:personal:mg-hotel-bot:9tJ5j93I:ckpt-step-72
ft:gpt-3.5-turbo-0125:personal:mg-hotel-bot:9tJ5j4qV
ft:gpt-3.5-turbo-0125:perso

In [36]:
# asking fine tuned model to give response of above question
# Create a chat completion request using the fine-tuned model
ashu_response = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0125:personal:ashu-hotel-bot:9tJJNpyu:ckpt-step-54",  # Ensure this is a string with your model ID
    messages=ashu_test_message,  # Use the prepared messages list
    temperature=0,
    max_tokens=400
)

# Print the response
print(ashu_response)
# original message
ashu_response["choices"][0]["message"]["content"]

{
  "id": "chatcmpl-9tVMrYw7Gycuawi7ibUhr3m6c6Ubl",
  "object": "chat.completion",
  "created": 1723015645,
  "model": "ft:gpt-3.5-turbo-0125:personal:ashu-hotel-bot:9tJJNpyu:ckpt-step-54",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "No problem! The WiFi password is \"Welcome123\". Let me know if you need help with anything else.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 49,
    "completion_tokens": 22,
    "total_tokens": 71
  },
  "system_fingerprint": null
}


'No problem! The WiFi password is "Welcome123". Let me know if you need help with anything else.'